In [16]:
from IPython.display import HTML
from base64 import b64encode
import os
import cv2
import random
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [2]:
# To Show a Video in Notebook
def Play_Video(filepath):
    video = open(filepath,'rb').read()
    src = 'data:video/mp4;base64,' + b64encode(video).decode()
    html = '<video width=640 muted controls autoplay loop><source src="%s" type="video/mp4"></video>' % src 
    return HTML(html)

In [3]:
# Classes Directories
NonViolenceVideos_Dir = "../data/NonViolence/"
ViolenceVideos_Dir = "../data/Violence/"

# Retrieve the list of all the video files present in the Class Directory.
NonViolence_files_names_list = os.listdir(NonViolenceVideos_Dir)
Violence_files_names_list = os.listdir(ViolenceVideos_Dir)

# Randomly select a video file from the Classes Directory.
Random_NonViolence_Video = random.choice(NonViolence_files_names_list)
Random_Violence_Video = random.choice(Violence_files_names_list)

In [4]:
Play_Video(f"{NonViolenceVideos_Dir}/{Random_NonViolence_Video}")

In [5]:
Play_Video(f"{ViolenceVideos_Dir}/{Random_Violence_Video}")

In [6]:
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64
 
# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 16
 
DATASET_DIR = "../data/"
 
CLASSES_LIST = ["NonViolence","Violence"]

In [7]:
def load_video_frames(video_path):
    """
    - Baca `seq_len` frame dari video di `path`, sampling merata di sepanjang durasi.
    - Center-crop setiap frame ke square
    - Resize ke 224 x 224
    - Konversi ke RGB dan normalisasi [0,1]
    """
    cap = cv2.VideoCapture(video_path)

    # 1. Dapatkan total frame
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # 2. Tentukan indeks frame yang akan diambil, sampling merata
    if total_frames < SEQUENCE_LENGTH:
        # Kalau video terlalu pendek, ambil semua dan ulangi terakhir sampai seq_len
        indices = list(range(total_frames)) + [total_frames - 1] * (SEQUENCE_LENGTH - total_frames)
    else:
        step = total_frames / SEQUENCE_LENGTH
        indices = [int(i * step) for i in range(SEQUENCE_LENGTH)]

    frames = []
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            # kalau gagal baca frame, ulangi frame terakhir
            frame = frames[-1] if frames else np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH, 3), np.uint8)
        # 3. Center‐crop ke square
        h, w = frame.shape[:2]
        m = min(h, w)
        y0 = (h - m) // 2
        x0 = (w - m) // 2
        crop = frame[y0:y0+m, x0:x0+m]
        # 4. Resize, RGB, normalisasi
        crop = cv2.resize(crop, (IMAGE_HEIGHT, IMAGE_WIDTH), interpolation=cv2.INTER_AREA)
        rgb  = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        frames.append(rgb.astype(np.float32) / 255.0)

    cap.release()
    return np.stack(frames, axis=0)


In [8]:
# Creating dataset
def create_dataset():
 
    features = []
    labels = []
    video_files_paths = []
    
    # Iterating through all the classes.
    for class_index, class_name in enumerate(CLASSES_LIST):
        
        print(f'Extracting Data of Class: {class_name}')
        
        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        
        # Iterate through all the files present in the files list.
        for file_name in files_list:
            
            # Get the complete video path.
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)
 
            # Extract the frames of the video file.
            frames = load_video_frames(video_file_path)
 
            # Check if the extracted frames are equal to the SEQUENCE_LENGTH specified.
            # So ignore the vides having frames less than the SEQUENCE_LENGTH.
            if len(frames) == SEQUENCE_LENGTH:
 
                # Append the data to their repective lists.
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)
 
    features = np.asarray(features)
    labels = np.array(labels)  

    return features, labels, video_files_paths

In [9]:
# Create the dataset.
features, labels, video_files_paths = create_dataset()

Extracting Data of Class: NonViolence
Extracting Data of Class: Violence


In [15]:
# convert labels into one-hot-encoded vectors
one_hot_encoded_labels = to_categorical(labels)

In [20]:
# Split the Data into Train ( 90% ) and Test Set ( 10% ).
X_train, X_test, y_train, y_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.1, shuffle = True, random_state = 42)


In [22]:
print(X_train.shape,y_train.shape )
print(X_test.shape, y_test.shape)

(1800, 16, 64, 64, 3) (1800, 2)
(200, 16, 64, 64, 3) (200, 2)


In [23]:
# Saving the extracted data
os.makedirs("../data/ProcessedData/", exist_ok=True)
np.save("../data/ProcessedData/X_train.npy",X_train)
np.save("../data/ProcessedData/X_test.npy",X_test)
np.save("../data/ProcessedData/y_train.npy",y_train)
np.save("../data/ProcessedData/y_test.npy",y_test)